<a href="https://colab.research.google.com/github/NitheeshMS/WorldBank_Data/blob/main/week4_animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing**

I have imported the datasets from my drive

In [2]:
import pandas as pd

# Load datasets
life=pd.read_csv('/content/drive/MyDrive/Data analyst internship/life_expectancy.csv')
fertility=pd.read_csv('/content/drive/MyDrive/Data analyst internship/fertility_rate.csv')
population=pd.read_csv('/content/drive/MyDrive/Data analyst internship/country_population.csv')
country=pd.read_csv('/content/drive/MyDrive/Data analyst internship/Metadata_Country.csv')

**Cleaning dataset**
Country dataset

In [ ]:
country.head()
country = country[['Country Code','Region']]
country= country.dropna()
country.isna().sum()

**Cleaning dataset**
Population dataset

In [ ]:
population.columns
# Drop 'Indicator Name' and 'Indicator Code' columns
population= population.drop(['Indicator Name', 'Indicator Code'],axis=1)
population = population[population['Country Name'] != 'Not classified']
population = population.dropna()
population.isna().sum()

**Cleaning dataset**
life_exceptancy dataset

In [ ]:
life = life.drop(['Indicator Name', 'Indicator Code'],axis=1)
life = life.dropna()
life.isna().sum()

**Cleaning dataset**
fertility_rate dataset

In [21]:
fertility= fertility.drop(['Indicator Name', 'Indicator Code'], axis=1)
fertility = fertility.dropna()
fertility.isna().sum()

**Melting the dataset**

Pandas melt was used to transform the dataset from a Wide format into a Long format. What is a Long format dataset? It's a dataset where the row is data of a combination of the variable and their values. In technical terms, we are unpivoting the dataset to acquire a dataset with fewer columns and longer rows.



In [69]:
# Melt fertility DataFrame
melted_fertility = fertility.melt(id_vars=['Country Name','Country Code'], var_name='Year', value_name='Fertility_rate')

# Melt life DataFrame
melted_life = life.melt(id_vars=['Country Name','Country Code'], var_name='Year', value_name='Life Expectancy')

# Melt population DataFrame
melted_population = population.melt(id_vars=['Country Name','Country Code'], var_name='Year', value_name='Population')

**Renaming the columns** -> so as to satisfy the Formate

In [87]:
# Rename columns in melted_country DataFrame
melted_fertility = melted_fertility.rename(columns={'Country Name': 'country_name','Country Code': 'country_code', 'Year': 'year','Fertility_rate':'fertility_rate' })

# Rename columns in melted_country DataFrame
melted_life = melted_life.rename(columns={'Country Name': 'country_name','Country Code': 'country_code', 'Year': 'year','Life Expectancy':'life_expectancy' })

# Rename columns in melted_country DataFrame
melted_population = melted_population.rename(columns={'Country Name': 'country_name','Country Code': 'country_code', 'Year': 'year','Population':'population' })

country= country.rename(columns={'Country Code': 'country_code', 'Region':'region' })


**Merging the datasets**

Along with merging I have performed the reodering of the columns after merging

In [ ]:
# Merge country with melted_fertility
merged = pd.merge(melted_fertility, country, left_on='country_code', right_on='country_code')


# Reorder columns in the merged dataset
merged = merged[['country_name', 'country_code', 'year', 'region', 'fertility_rate']]

# Merge country and fertility datasets
merged_country = pd.merge(merged, melted_population, left_on=['country_name', 'country_code', 'year'], right_on=['country_name', 'country_code', 'year'], how='left')

# Reorder columns
merged_country = merged_country[['country_name', 'country_code', 'year', 'region', 'fertility_rate', 'population']]

# Merge the existing merged dataset with melted_life
final_merged_dataset = pd.merge(merged_country, melted_life, left_on=['country_name', 'country_code', 'year'], right_on=['country_name', 'country_code', 'year'], how='left')

# Reorder columns
final_merged_dataset = final_merged_dataset[['country_name', 'country_code', 'year', 'region', 'fertility_rate', 'population', 'life_expectancy']]

**Rounding off to single decimal place**

In [106]:
# Round columns to 1 decimal point
final_merged_dataset['fertility_rate'] = final_merged_dataset['fertility_rate'].round(1)
final_merged_dataset['population'] = final_merged_dataset['population'].round(1)
final_merged_dataset['life_expectancy'] = final_merged_dataset['life_expectancy'].round(1)

*After the mergeing was done  , there was still some **NaN**(null vlaues) present so i had to drop them to clean the dataset*

In [117]:
final_merged_dataset.dropna(subset=['population', 'life_expectancy'], inplace=True)


Finally the **animation**

I have used polty.express inorde the get the animated graph and you can see the code below and the parameters considered for the graphs

In [121]:
import plotly.express as px

# Replace 'final_merged_dataset' with actual DataFrame
df = final_merged_dataset

# Create the scatter plot animation
fig = px.scatter(df,
                 x='fertility_rate',
                 y='life_expectancy',
                 size='population',
                 size_max=50,
                 hover_name='country_name',
                 color='region',
                 animation_frame='year',
                 animation_group='country_name',
                 labels={'region':'Region'},
                 range_x=[0, 10],
                 range_y=[10, 90]
                )

# Update axis labels and title
fig.update_layout(

    xaxis_title='Fertility Rate',
    yaxis_title='Life Expectancy',
    title='Fertility Rate and Life Expectancy Over Time'
)

# Show the animation
fig.show()
